In [ ]:
import numpy as np
import pandas as pd

label_dic = {
    0: 'healthy', 
    1: 'scab',
    2: 'rust',
    3: 'frog_eye_leaf_spot',
    4: 'complex', 
    5: 'powdery_mildew'
}

In [ ]:
!apt install ../input/pyturbojpeg/libturbojpeg_1.4.2-0ubuntu3.4_amd64.deb
!pip install ../input/pyturbojpeg/PyTurboJPEG-1.4.1

In [ ]:
import matplotlib.pyplot as plt
import albumentations as A
import cv2, torch
import torchvision.transforms as transforms
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from turbojpeg import TurboJPEG

device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

#######################################

from albumentations.pytorch import ToTensor

def transform_valid():
    
    augmentation_pipeline = A.Compose(
        [
            A.SmallestMaxSize(224),
            A.RandomCrop(224, 224),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
                ),
            ToTensor() 
        ],
        p = 1
    )
    return lambda img:augmentation_pipeline(image=np.array(img))['image']

######################################

jpeg_reader = TurboJPEG()

def read_img(img):
    with open(img, "rb") as f:
        return jpeg_reader.decode(f.read(), 0) 

In [ ]:
import sys

sys.path.append("../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master")

from efficientnet_pytorch import model as enet

model = enet.EfficientNet.from_name('efficientnet-b0')

import torch.nn as nn

model._fc = nn.Linear(in_features=model._fc.in_features, out_features=6)

model.load_state_dict(torch.load('../input/pytorch-efficientnet/best_model.pth'))

model = model.to(device)

In [ ]:
from glob import glob
classes = 6
valid_image_list = glob('../input/plant-pathology-2021-fgvc8/test_images/*.jpg')

def to_lab(preds):
    return ((preds > 0) + torch.nn.functional.one_hot(preds.argmax(1), 6) != 0).long()

model.eval()
predict_list = []
image_name_list = []
for i, image in tqdm(enumerate(valid_image_list)) :
    image_name = image[48:]
    
    img = read_img(image)
    img = transform_valid()(img)
    
    result_list = torch.FloatTensor(np.zeros((classes))).to(device)
    img = img.to(device)
    img = img.reshape(-1, 3, 224, 224)
    with torch.set_grad_enabled(False):
        predict = model(img)
    #predict = predict.reshape(-1)
    #result_list += (predict > 0).nonzero().reshape(-1)
    predict_list.append(list(to_lab(predict).reshape(-1).nonzero().reshape(-1).cpu().numpy()))
    #predict_list.append(result_list)
    image_name_list.append(image_name)
    
#predict_list = np.array(predict_list)
image_name_list = np.array(image_name_list)
print(image_name_list)

submission_df = pd.DataFrame()
submission_df['image'] = image_name_list
submission_df['label_id'] = predict_list
submission_df['labels'] = submission_df['label_id'].apply(lambda x: " ".join([label_dic[i] for i in x]))
del submission_df['label_id']
submission_df.head()

In [ ]:
submission_df.to_csv("submission.csv", index = False)